In [1]:
# favorite example

# cam2, start_index = 20, end_index = 23

# point №40
# it corresponds to instance 53 (image 20)
# it corresponds to instance 45 (image 21)
# it corresponds to instance 53 (image 22)

In [2]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [3]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
labeled_pcds, colored_pcds, coo_matrix = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [4]:
len(coo_matrix)

3

In [5]:
# point №40
# it corresponds to instance 53 (image 22)
print(coo_matrix[2].data[40])

53.0


In [6]:
len(coo_matrix[0].data)

277369

In [7]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views_data

data1 = construct_coo_matrix_for_multiple_views_data(coo_matrix)

In [8]:
# point40 corresponds to instance 45 (image 21). I want to check it in the result data
# coo_matrix[0].data = 277369
# 277369 + 40 = 277409 => index of point40 (image 21) in the result data
data1[277409]

45.0

In [9]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views

coo_result = construct_coo_matrix_for_multiple_views(coo_matrix, 20)

In [10]:
csr_multiple_views = coo_result.tocsr()

In [11]:
# image_index = 2 (image 22) and point №40. expected value = 53
csr_multiple_views[2, 40]

53.0

In [12]:
# image_index = 1 (image 21) and point №40. expected value = 45
csr_multiple_views[1, 40]

45.0

In [13]:
csr_multiple_views[0, 0]

0.0

In [14]:
csr_multiple_views[[0], :].nnz

277369

In [15]:
coo = csr_multiple_views[[0], :]

In [16]:
(csr_multiple_views[[1], [40]])[0]

45.0

In [17]:
from mapping.sparse_matrix_utils import visualize_labeled_clouds
pcd = visualize_labeled_clouds(coo_result, 21, 20, 23, sm)

In [18]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [15]:
enc.fit(coo_result.toarray())

OneHotEncoder(handle_unknown='ignore')

In [20]:
enc.categories_[40]

array([45., 53.])